In [3]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
 
# Setting model parameters
C = 1.0
n_splits = 5
 
output_file = f'model_C={C}.bin'
 
# Data preparation
df = pd.read_csv('data-week-3.csv')
 
df.columns = df.columns.str.lower().str.replace(' ', '_')
 
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
 
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')
 
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)
 
df.churn = (df.churn == 'yes').astype(int)
 
# Data splitting
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
 
numerical = ['tenure', 'monthlycharges', 'totalcharges']
 
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']
 
# Training
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
 
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
 
    model = LogisticRegression(C=C, max_iter=5000)
    model.fit(X_train, y_train)
 
    return dv, model
 
def predict(df, dv, model):
     dicts = df[categorical + numerical].to_dict(orient='records')
 
     X = dv.fit_transform(dicts)
     y_pred = model.predict_proba(X)[:,1]
 
     return y_pred
 
# Validation
print(f'doing validation with C={C}')
 
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)  
     
scores = []
 
fold = 0
 
for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
 
    y_train = df_train.churn.values
    y_val = df_val.churn.values
 
    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)
 
    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)
 
    print(f'auc on fold {fold} is {auc}')
    fold += 1
 
print('validation result:')
print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))
# Output: C=1.0 0.841 +- 0.008
 
# Train the final model
print('train the final model')
 
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)
y_test = df_test.churn.values
 
auc = roc_auc_score(y_test, y_pred)
 
print(f'auc={auc}')
# Output: 0.8572386167896259
 
# Saving the model with Pickle
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)
 
print(f'the model is saved to {output_file}')

doing validation with C=1.0
auc on fold 0 is 0.8443375120691413
auc on fold 1 is 0.8449977651966627
auc on fold 2 is 0.8333012241621514
auc on fold 3 is 0.8347768913474451
auc on fold 4 is 0.8518166985660947
validation result:
C=1.0 0.842 +- 0.007
train the final model
auc=0.8584032088573997
the model is saved to model_C=1.0.bin
